In [8]:
#0.Import Libraries
import pandas as pd
import numpy as np
import requests
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pybit.unified_trading import HTTP
session = HTTP(testnet = False)

#1.Model Params & Endpoints
Symbol = 'ASTERUSDT'
Interval_Aster = '1m'
Interval_bybit = 1
Limit = 1000

#2.Request Params
def Get_Kline_Aster():
    Ep_Aster_Kline = 'https://fapi.asterdex.com/fapi/v1/klines'
    Aster_Kline = {'symbol':Symbol, 'interval':Interval_Aster, 'limit':Limit}
    raw_Aster_Kline = requests.get(Ep_Aster_Kline, Aster_Kline)
    json_Aster_Kline = raw_Aster_Kline.json()
    df = pd.DataFrame([(row[0], float(row[1]), float(row[2]), float(row[3]), float(row[4])) for row in json_Aster_Kline], columns=["timestamp", "O_Ast", "H_Ast", "L_Ast", "C_Ast"])
    return df

def Get_Kline_bybit():
    raw_bybit_Kline = session.get_kline(
        category = 'linear',
        symbol = Symbol,
        interval = Interval_bybit,
        limit = 1000
    )
    list = raw_bybit_Kline['result']['list']
    df = pd.DataFrame([(row[0], float(row[1]), float(row[2]), float(row[3]), float(row[4])) for row in list], columns=["timestamp", "O_byb", "H_byb", "L_byb", "C_byb"])
    return df

#3.Get Kline and Merge
df_Aster = Get_Kline_Aster()
df_Bybit = Get_Kline_bybit()
df_Bybit["timestamp"] = pd.to_datetime(df_Bybit["timestamp"].astype("int64"), unit="ms")
df_Aster["timestamp"] = pd.to_datetime(df_Aster["timestamp"].astype("int64"), unit="ms")
df_merged = pd.merge(df_Bybit, df_Aster, on="timestamp", how="inner")

#4.Calculate Spread
df_merged['Aster_diff'] = ((df_merged['C_Ast'] / df_merged['C_byb']) - 1)

#5.Draw in graph
df_merged["timestamp_fmt"] = df_merged["timestamp"].dt.strftime("%m/%d %H:%M")

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    row_heights=[0.25, 0.375, 0.375],  # 上段=25%, 中下段=37.5%ずつ
    vertical_spacing=0.05,
    subplot_titles=("Spread (Aster/Bybit - 1)", "Bybit OHLC", "Aster OHLC")
)

# --- 1段目：Spread（％表記）
fig.add_trace(
    go.Scatter(
        x=df_merged["timestamp"],
        y=df_merged["Aster_diff"],
        mode="lines",
        name="Spread",
        hovertemplate="%{x|%m/%d %H:%M}<br>Spread: %{y:.3%}<extra></extra>"
    ),
    row=1, col=1
)
fig.add_hline(y=0, line_dash="dot", row=1, col=1)

# --- 2段目：Bybitローソク足
fig.add_trace(
    go.Candlestick(
        x=df_merged["timestamp"],
        open=df_merged["O_byb"], high=df_merged["H_byb"],
        low=df_merged["L_byb"], close=df_merged["C_byb"],
        name="Bybit",
        increasing_line_color="blue", decreasing_line_color="lightblue"
    ),
    row=2, col=1
)

# --- 3段目：Asterローソク足
fig.add_trace(
    go.Candlestick(
        x=df_merged["timestamp"],
        open=df_merged["O_Ast"], high=df_merged["H_Ast"],
        low=df_merged["L_Ast"], close=df_merged["C_Ast"],
        name="Aster",
        increasing_line_color="red", decreasing_line_color="pink"
    ),
    row=3, col=1
)

# --- 体裁
fig.update_layout(
    title="Bybit vs Aster — Spread & OHLC",
    hovermode="x unified",
    height=900,  # ← グラフ全体の高さを増やす
    yaxis=dict(title="Spread (%)", tickformat=".3%"),
    yaxis2=dict(title="Bybit Price (USDT)"),
    yaxis3=dict(title="Aster Price (USDT)"),
    xaxis=dict(showticklabels=False),          # 上段は非表示
    xaxis2=dict(showticklabels=False),         # 中段も非表示
    xaxis3=dict(tickformat="%m/%d %H:%M"),     # 下段のみ表示
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=False,
    xaxis3_rangeslider_visible=False
)

fig.show()

def backtest_spread_premium_flip(
    df: pd.DataFrame,
    enter_thresh: float = 0.005,     # 0.10%
    exit_thresh:  float = 0.0001,    # 0.05%
    fee_per_leg:  float = 0.00045,    # 0.09% per leg per trade
    notional:     float = 10_000.0,  # USD/USDT per leg
):
    """
    ルール：
      - 判定はバーiのCで行い、約定はバーi+1のO
      - プレミアム (C_Ast/C_byb - 1) >= enter_thresh で「建て」：
            Aster を売り、Bybit を買い（同額）
      - プレミアム <= -exit_thresh で「解消」：
            Aster 買い戻し、Bybit 売り（同額）
      - 最終足で未解消はノークローズ（保守的）
    """
    d = df.sort_values("timestamp").reset_index(drop=True).copy()

    # 安全チェック
    req_cols = ["timestamp","O_byb","H_byb","L_byb","C_byb","O_Ast","H_Ast","L_Ast","C_Ast","Aster_diff"]
    missing = [c for c in req_cols if c not in d.columns]
    if missing:
        raise ValueError(f"columns missing: {missing}")

    position = 0   # 0=ノーポジ, 1= ペア建て（Asterショート/Bybitロング）
    entry_idx = None
    qty_byb = qty_ast = 0.0
    entry_prices = {}

    trades = []  # 約定ログ

    # i でCを見て、i+1 で約定するため、レンジは 0..len-2
    for i in range(len(d) - 1):
        prem = d.loc[i, "Aster_diff"]
        ts_c = d.loc[i, "timestamp"]
        # 次足の約定価格
        ts_o = d.loc[i+1, "timestamp"]
        o_byb = d.loc[i+1, "O_byb"]
        o_ast = d.loc[i+1, "O_Ast"]

        if position == 0:
            # エントリー条件：プレミアムが閾値以上
            if prem >= enter_thresh:
                # 同額USDで両建て
                qty_byb = notional / o_byb              # ロング数量
                qty_ast = notional / o_ast              # ショート数量

                # 片道・片側の手数料 × 2レグ
                entry_fee = notional * fee_per_leg * 2

                entry_idx = i+1
                entry_prices = {"O_byb": o_byb, "O_Ast": o_ast}
                position = 1

                trades.append({
                    "type": "ENTRY",
                    "signal_time": ts_c,       # シグナル判定の時刻（Close）
                    "exec_time": ts_o,         # 実約定の時刻（次のOpen）
                    "price_byb": o_byb,
                    "price_ast": o_ast,
                    "qty_byb": qty_byb,
                    "qty_ast_short": qty_ast,
                    "fee": entry_fee,
                })

        else:
            # クローズ条件：ディスカウントが閾値以上（＝プレミアム <= -exit_thresh）
            if prem <= -exit_thresh:
                # 退出時の価格（次バーOpen）
                close_byb = o_byb
                close_ast = o_ast

                # PnL計算：Bybitロング利益 - Asterショート損益
                pnl_byb = qty_byb * (close_byb - entry_prices["O_byb"])
                pnl_ast = qty_ast * (entry_prices["O_Ast"] - close_ast)  # ショートは逆
                gross_pnl = pnl_byb + pnl_ast

                exit_fee = notional * fee_per_leg * 2
                net_pnl = gross_pnl - (exit_fee)

                # エントリー時点で払った手数料も差し引いて、トータル損益を記録
                # （ENTRY行で fee を記録済みなので、ここでは exit分だけ引いて、
                #   trade毎の合計は別途集計時にENTRY+EXITを合算して評価でもOK）
                trades.append({
                    "type": "EXIT",
                    "signal_time": d.loc[i, "timestamp"],
                    "exec_time": ts_o,
                    "price_byb": close_byb,
                    "price_ast": close_ast,
                    "qty_byb": qty_byb,
                    "qty_ast_cover": qty_ast,
                    "gross_pnl": gross_pnl,
                    "fee": exit_fee,
                    "net_pnl_excluding_entry_fee": net_pnl,
                    "entry_index": entry_idx,
                })

                # リセット
                position = 0
                entry_idx = None
                qty_byb = qty_ast = 0.0
                entry_prices = {}

    trades_df = pd.DataFrame(trades)

    # トレード集計（ENTRY/EXITをマッチさせて1トレード単位にする）
    # ENTRYとEXITは交互に出るはずなので、ペアリング
    results = []
    running = {}
    for _, row in trades_df.iterrows():
        if row["type"] == "ENTRY":
            running = {
                "entry_time": row["exec_time"],
                "entry_fee": row["fee"],
                "entry_price_byb": row["price_byb"],
                "entry_price_ast": row["price_ast"],
                "qty_byb": row["qty_byb"],
                "qty_ast": row["qty_ast_short"],
            }
        else:
            # EXIT
            if running:
                gross = row["gross_pnl"]
                exit_fee = row["fee"]
                total_fee = running["entry_fee"] + exit_fee
                net = gross - total_fee
                results.append({
                    "entry_time": running["entry_time"],
                    "exit_time": row["exec_time"],
                    "entry_price_byb": running["entry_price_byb"],
                    "entry_price_ast": running["entry_price_ast"],
                    "exit_price_byb": row["price_byb"],
                    "exit_price_ast": row["price_ast"],
                    "qty_byb": running["qty_byb"],
                    "qty_ast": running["qty_ast"],
                    "gross_pnl": gross,
                    "fees_total": total_fee,
                    "net_pnl": net,
                })
                running = {}

    results_df = pd.DataFrame(results)
    summary = {}
    if not results_df.empty:
        summary = {
            "n_trades": len(results_df),
            "gross_pnl": results_df["gross_pnl"].sum(),
            "fees_total": results_df["fees_total"].sum(),
            "net_pnl": results_df["net_pnl"].sum(),
            "win_rate": (results_df["net_pnl"] > 0).mean() if len(results_df) else np.nan,
            "avg_net_per_trade": results_df["net_pnl"].mean() if len(results_df) else np.nan,
        }
    return results_df, trades_df, summary

# 使い方：
results_df, trades_df, summary = backtest_spread_premium_flip(
    df_merged,
    enter_thresh=0.002,   # 0.10% 以上で建て
    exit_thresh=0.0001,    # -0.05% 以下で解消
    fee_per_leg=0.00045,    # 0.09% 片道・片側
    notional=1000
)

print(summary)
results_df

fee_taker = ((0.00035*2 + 0.00055*2)/2) * 100
print(f'taker:{fee_taker}')
fee_maker = ((0.0001*2 + 0.0002*2)/2) * 100
print(f'maker:{fee_maker}')

{'n_trades': 6, 'gross_pnl': 15.395457753207358, 'fees_total': 10.8, 'net_pnl': 4.595457753207358, 'win_rate': 1.0, 'avg_net_per_trade': 0.7659096255345598}
taker:0.09
maker:0.030000000000000002


In [9]:
def backtest_spread_with_fixed_qty_and_capital(
    df: pd.DataFrame,
    enter_thresh: float = 0.002,      # プレミアムでMain IN
    exit_thresh:  float = 0.0001,     # ディスカウントでMain OUT→Bias
    fee_per_leg:  float = 0.00045,    # 片道・片側手数料
    # --- 資金＆レバ ---
    byb_initial_capital: float = 10_000.0,
    ast_initial_capital: float = 10_000.0,
    byb_leverage: float = 3.0,
    ast_leverage: float = 3.0,
    # 固定枚数（指定が無ければ初回ENTRYで 初期資金×レバ/価格 で決定）
    byb_fixed_qty: float | None = None,
    ast_fixed_qty: float | None = None,
):
    d = df.sort_values("timestamp").reset_index(drop=True).copy()
    req = ["timestamp","O_byb","C_byb","O_Ast","C_Ast","Aster_diff"]
    miss = [c for c in req if c not in d.columns]
    if miss:
        raise ValueError(f"columns missing: {miss}")

    # 状態: 1=Bias(Aster LONG / Bybit SHORT), 2=Main(Aster SHORT / Bybit LONG)
    state = 0
    qty_byb = byb_fixed_qty
    qty_ast = ast_fixed_qty

    cap_byb = byb_initial_capital
    cap_ast = ast_initial_capital

    entry_byb = entry_ast = None
    side_byb = side_ast = None

    logs = []            # ENTRY/EXITの生ログ
    cap_history = []     # EXIT時の資金スナップショット

    # ENTRYで記録しておき、EXITでNetを計算
    running = {"BIAS": None, "MAIN": None}
    results_rows = []    # 1トレード＝ENTRY→EXITのまとまり

    def fee_amt(price, qty): return price * qty * fee_per_leg

    def open_bias(i_next):
        nonlocal state, qty_byb, qty_ast, cap_byb, cap_ast, entry_byb, entry_ast, side_byb, side_ast, running
        o_byb = d.loc[i_next,"O_byb"]; o_ast = d.loc[i_next,"O_Ast"]

        if qty_byb is None:
            qty_byb = float((byb_initial_capital * byb_leverage) / o_byb)
        if qty_ast is None:
            qty_ast = float((ast_initial_capital * ast_leverage) / o_ast)

        # 手数料（建玉）
        fee_open_byb = fee_amt(o_byb, qty_byb)
        fee_open_ast = fee_amt(o_ast, qty_ast)
        cap_byb -= fee_open_byb
        cap_ast -= fee_open_ast

        entry_byb, entry_ast = o_byb, o_ast
        side_byb, side_ast = "short", "long"
        state = 1

        logs.append({"type":"ENTRY_BIAS","exec_time":d.loc[i_next,"timestamp"],
                     "price_byb":o_byb,"price_ast":o_ast,"qty_byb":qty_byb,"qty_ast":qty_ast,
                     "fee_byb": fee_open_byb, "fee_ast": fee_open_ast})

        # ENTRY情報を保持（Bias）
        running["BIAS"] = {
            "entry_time": d.loc[i_next,"timestamp"],
            "fee_open_byb": fee_open_byb,
            "fee_open_ast": fee_open_ast
        }

    def open_main(i_next):
        nonlocal state, cap_byb, cap_ast, entry_byb, entry_ast, side_byb, side_ast, running
        o_byb = d.loc[i_next,"O_byb"]; o_ast = d.loc[i_next,"O_Ast"]

        fee_open_byb = fee_amt(o_byb, qty_byb)
        fee_open_ast = fee_amt(o_ast, qty_ast)
        cap_byb -= fee_open_byb
        cap_ast -= fee_open_ast

        entry_byb, entry_ast = o_byb, o_ast
        side_byb, side_ast = "long", "short"
        state = 2

        logs.append({"type":"ENTRY_MAIN","exec_time":d.loc[i_next,"timestamp"],
                     "price_byb":o_byb,"price_ast":o_ast,"qty_byb":qty_byb,"qty_ast":qty_ast,
                     "fee_byb": fee_open_byb, "fee_ast": fee_open_ast})

        # ENTRY情報を保持（Main）
        running["MAIN"] = {
            "entry_time": d.loc[i_next,"timestamp"],
            "fee_open_byb": fee_open_byb,
            "fee_open_ast": fee_open_ast
        }

    def close_bias(i_sig, i_next):
        nonlocal cap_byb, cap_ast, entry_byb, entry_ast, side_byb, side_ast, running
        o_byb = d.loc[i_next,"O_byb"]; o_ast = d.loc[i_next,"O_Ast"]

        pnl_byb = (entry_byb - o_byb) * qty_byb        # Bybit short
        pnl_ast = (o_ast - entry_ast) * qty_ast        # Aster long
        fee_close_byb = fee_amt(o_byb, qty_byb)
        fee_close_ast = fee_amt(o_ast, qty_ast)

        cap_byb += pnl_byb - fee_close_byb
        cap_ast += pnl_ast - fee_close_ast

        logs.append({"type":"EXIT_BIAS","signal_time":d.loc[i_sig,"timestamp"],
                     "exec_time":d.loc[i_next,"timestamp"],"price_byb":o_byb,"price_ast":o_ast,
                     "pnl_byb":pnl_byb,"pnl_ast":pnl_ast,
                     "fee_byb": fee_close_byb, "fee_ast": fee_close_ast,
                     "cap_byb":cap_byb,"cap_ast":cap_ast})

        # トレード集計（Bias）
        ent = running["BIAS"];  running["BIAS"] = None
        net = (pnl_byb + pnl_ast) - (ent["fee_open_byb"] + ent["fee_open_ast"] + fee_close_byb + fee_close_ast)
        results_rows.append({
            "side":"BIAS",
            "entry_time": ent["entry_time"],
            "exit_time": d.loc[i_next,"timestamp"],
            "gross_pnl": pnl_byb + pnl_ast,
            "fees_total": ent["fee_open_byb"] + ent["fee_open_ast"] + fee_close_byb + fee_close_ast,
            "net_pnl": net
        })

        entry_byb = entry_ast = None
        side_byb = side_ast = None
        cap_history.append({"timestamp":d.loc[i_next,"timestamp"], "cap_byb":cap_byb, "cap_ast":cap_ast})

    def close_main(i_sig, i_next):
        nonlocal cap_byb, cap_ast, entry_byb, entry_ast, side_byb, side_ast, running
        o_byb = d.loc[i_next,"O_byb"]; o_ast = d.loc[i_next,"O_Ast"]

        pnl_byb = (o_byb - entry_byb) * qty_byb        # Bybit long
        pnl_ast = (entry_ast - o_ast) * qty_ast        # Aster short
        fee_close_byb = fee_amt(o_byb, qty_byb)
        fee_close_ast = fee_amt(o_ast, qty_ast)

        cap_byb += pnl_byb - fee_close_byb
        cap_ast += pnl_ast - fee_close_ast

        logs.append({"type":"EXIT_MAIN","signal_time":d.loc[i_sig,"timestamp"],
                     "exec_time":d.loc[i_next,"timestamp"],"price_byb":o_byb,"price_ast":o_ast,
                     "pnl_byb":pnl_byb,"pnl_ast":pnl_ast,
                     "fee_byb": fee_close_byb, "fee_ast": fee_close_ast,
                     "cap_byb":cap_byb,"cap_ast":cap_ast})

        # トレード集計（Main）
        ent = running["MAIN"];  running["MAIN"] = None
        net = (pnl_byb + pnl_ast) - (ent["fee_open_byb"] + ent["fee_open_ast"] + fee_close_byb + fee_close_ast)
        results_rows.append({
            "side":"MAIN",
            "entry_time": ent["entry_time"],
            "exit_time": d.loc[i_next,"timestamp"],
            "gross_pnl": pnl_byb + pnl_ast,
            "fees_total": ent["fee_open_byb"] + ent["fee_open_ast"] + fee_close_byb + fee_close_ast,
            "net_pnl": net
        })

        entry_byb = entry_ast = None
        side_byb = side_ast = None
        cap_history.append({"timestamp":d.loc[i_next,"timestamp"], "cap_byb":cap_byb, "cap_ast":cap_ast})

    # --- シミュレーション ---
    if len(d) < 2:
        return pd.DataFrame(), pd.DataFrame(), {}, pd.DataFrame()

    # 初回: i=1 の Open で Bias 建て
    open_bias(1)

    # i=1..n-2: Closeで判定 → i+1 Openで約定
    for i in range(1, len(d)-1):
        prem = d.loc[i,"Aster_diff"]
        if state == 1 and prem >= enter_thresh:
            close_bias(i, i+1)
            open_main(i+1)
        elif state == 2 and prem <= -exit_thresh:
            close_main(i, i+1)
            open_bias(i+1)

    trades_df = pd.DataFrame(logs)
    results_df = pd.DataFrame(results_rows)
    cap_history_df = pd.DataFrame(cap_history).sort_values("timestamp").reset_index(drop=True)

    summary = {
        "byb_final_capital": cap_byb,
        "ast_final_capital": cap_ast,
        "byb_return_pct": (cap_byb / byb_initial_capital - 1.0) * 100.0,
        "ast_return_pct": (cap_ast / ast_initial_capital - 1.0) * 100.0,
        "n_trades_main": (results_df["side"]=="MAIN").sum() if not results_df.empty else 0,
        "n_trades_bias": (results_df["side"]=="BIAS").sum() if not results_df.empty else 0,
        "net_pnl_total": results_df["net_pnl"].sum() if not results_df.empty else 0.0,
    }
    return results_df, trades_df, summary, cap_history_df




def plot_upnl_per_leg(timeline_df, height=380):
    t = timeline_df.copy()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=t["timestamp"], y=t["byb_upnl_pct"], mode="lines",
        name="Bybit uPnL (%)"
    ))
    fig.add_trace(go.Scatter(
        x=t["timestamp"], y=t["ast_upnl_pct"], mode="lines",
        name="Aster uPnL (%)"
    ))
    fig.add_hline(y=0, line_dash="dot")
    fig.update_layout(title="Unrealized PnL per Leg (%)",
                      yaxis_title="uPnL (%)", height=height,
                      xaxis=dict(tickformat="%m/%d %H:%M"))
    fig.show()


def plot_spread_ohlc_with_markers(df_merged, trades_df, height=900):
    d = df_merged.sort_values("timestamp").reset_index(drop=True)

    byb_entry_long = trades_df[trades_df["type"]=="ENTRY_MAIN"]
    byb_exit_long  = trades_df[trades_df["type"]=="EXIT_MAIN"]
    byb_entry_short = trades_df[trades_df["type"]=="ENTRY_BIAS"]
    byb_exit_short  = trades_df[trades_df["type"]=="EXIT_BIAS"]

    ast_entry_long = trades_df[trades_df["type"]=="ENTRY_BIAS"]
    ast_exit_long  = trades_df[trades_df["type"]=="EXIT_BIAS"]
    ast_entry_short = trades_df[trades_df["type"]=="ENTRY_MAIN"]
    ast_exit_short  = trades_df[trades_df["type"]=="EXIT_MAIN"]

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        row_heights=[0.25,0.375,0.375], vertical_spacing=0.05,
                        subplot_titles=("Spread (Aster/Bybit - 1)", "Bybit OHLC + Trades", "Aster OHLC + Trades"))

    # Spread
    fig.add_trace(go.Scatter(x=d["timestamp"], y=d["Aster_diff"], mode="lines", name="Spread",
                             hovertemplate="%{x|%m/%d %H:%M}<br>Spread: %{y:.3%}<extra></extra>"), row=1, col=1)
    fig.add_hline(y=0, line_dash="dot", row=1, col=1)

    # Bybit
    fig.add_trace(go.Candlestick(x=d["timestamp"], open=d["O_byb"], high=d["H_byb"], low=d["L_byb"], close=d["C_byb"],
                                 name="Bybit", increasing_line_color="blue", decreasing_line_color="lightblue"),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=byb_entry_long["exec_time"], y=byb_entry_long["price_byb"], mode="markers",
                             name="Bybit LONG entry", marker_symbol="triangle-up", marker_size=10), row=2, col=1)
    fig.add_trace(go.Scatter(x=byb_exit_long["exec_time"], y=byb_exit_long["price_byb"], mode="markers",
                             name="Bybit LONG exit", marker_symbol="x", marker_size=10), row=2, col=1)
    fig.add_trace(go.Scatter(x=byb_entry_short["exec_time"], y=byb_entry_short["price_byb"], mode="markers",
                             name="Bybit SHORT entry", marker_symbol="triangle-down", marker_size=10), row=2, col=1)
    fig.add_trace(go.Scatter(x=byb_exit_short["exec_time"], y=byb_exit_short["price_byb"], mode="markers",
                             name="Bybit SHORT exit", marker_symbol="x-thin", marker_size=10), row=2, col=1)

    # Aster
    fig.add_trace(go.Candlestick(x=d["timestamp"], open=d["O_Ast"], high=d["H_Ast"], low=d["L_Ast"], close=d["C_Ast"],
                                 name="Aster", increasing_line_color="red", decreasing_line_color="pink"),
                  row=3, col=1)
    fig.add_trace(go.Scatter(x=ast_entry_long["exec_time"], y=ast_entry_long["price_ast"], mode="markers",
                             name="Aster LONG entry", marker_symbol="triangle-up", marker_size=10), row=3, col=1)
    fig.add_trace(go.Scatter(x=ast_exit_long["exec_time"], y=ast_exit_long["price_ast"], mode="markers",
                             name="Aster LONG exit", marker_symbol="x", marker_size=10), row=3, col=1)
    fig.add_trace(go.Scatter(x=ast_entry_short["exec_time"], y=ast_entry_short["price_ast"], mode="markers",
                             name="Aster SHORT entry", marker_symbol="triangle-down", marker_size=10), row=3, col=1)
    fig.add_trace(go.Scatter(x=ast_exit_short["exec_time"], y=ast_exit_short["price_ast"], mode="markers",
                             name="Aster SHORT exit", marker_symbol="x-thin", marker_size=10), row=3, col=1)

    fig.update_layout(height=height, hovermode="x unified",
                      yaxis=dict(title="Spread (%)", tickformat=".3%"),
                      yaxis2=dict(title="Bybit Price (USDT)"),
                      yaxis3=dict(title="Aster Price (USDT)"),
                      xaxis3=dict(tickformat="%m/%d %H:%M"),
                      xaxis_rangeslider_visible=False,
                      xaxis2_rangeslider_visible=False,
                      xaxis3_rangeslider_visible=False)
    fig.show()


# ===== 実行例 =====
results_df, trades_df, summary, cap_hist = backtest_spread_with_fixed_qty_and_capital(
    df_merged,
    enter_thresh=0.0020,
    exit_thresh=0.0001,
    fee_per_leg=0.00045,
    byb_initial_capital=100,
    ast_initial_capital=100,
    byb_leverage=3.0,
    ast_leverage=3.0,
    byb_fixed_qty = 200,
    ast_fixed_qty = 200
)
print(summary)

plot_spread_ohlc_with_markers(df_merged, trades_df, height=1200)



{'byb_final_capital': 120.87884899999995, 'ast_final_capital': 81.45491599999993, 'byb_return_pct': 20.878848999999946, 'ast_return_pct': -18.545084000000077, 'n_trades_main': 6, 'n_trades_bias': 6, 'net_pnl_total': 2.6799769999998477}
